## Feature Engeneering 

In [1]:
import pandas as pd
pd.options.display.max_columns = 25



In [2]:
import sys 
sys.path.append("../..")
from dpa_2021.src.utils.utils import *

In [3]:
df= pd.read_csv('../data/Food_Inspections.csv')
df.head()

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,Location
0,2473009,JENI'S SPLENDID ICE CREAMS,JENI'S SPLENDID ICE CREAMS,2767260.0,Restaurant,Risk 2 (Medium),2512 N MILKWAUKEE AVE,CHICAGO,IL,60647.0,01/22/2021,License,Pass,NaN,NaN,NaN,NaN
1,2457202,PI18EN,PI18EN,2626471.0,Restaurant,Risk 1 (High),1519 W 18TH ST,CHICAGO,IL,60608.0,11/30/2020,Non-Inspection,No Entry,NaN,41.857714,-87.664542,"(-87.66454238198166, 41.8577138739859)"
2,2453552,CLAMP DOWN BURGERS,NaN,2749943.0,NaN,Risk 2 (Medium),1742 W DIVISION ST,CHICAGO,IL,60622.0,10/20/2020,License,No Entry,NaN,41.903387,-87.671740,"(-87.67174026586648, 41.903386755553484)"
3,2401262,Fleming Branch,Fleming Branch,23462.0,School,Risk 1 (High),4918 W 64th ST,CHICAGO,IL,60638.0,09/16/2020,Canvass,Pass,NaN,41.776651,-87.745551,"(-87.74555107977419, 41.7766510612827)"
4,2387142,Pritzker,Pritzker,25871.0,School,Risk 1 (High),2009 W Schiller ST,CHICAGO,IL,60622.0,09/14/2020,Canvass,Pass,NaN,41.907025,-87.677796,"(-87.67779554588037, 41.907025004582245)"


## Nos quedamos con las siguientes variables :

(Todas son variables son  categóricas )

- Risk 
- Facility type (agrupar con regex por nombres similares de categorías; y categorías top y other) 
- City (simplificada, chicago y other) (regex para chicago) 
- Inspection type (agrupar con regex y categorías top pero respetar cateogorías más imoprtantes, sobre todo, REINSPECTION, no agrupar ) .  
- Violations (regex, buscar los códigos vinculados con violación grave, categorías "tiene violación grave o no").


## Descartar variables:

 - inspection ID
 - AKA name
 - DBA name
 - Licence #
 - Address
 - 
 - State
 - Latitude
 - Longitud
 - Location

In [14]:
df['zip'].isna().sum()

52

 - City, convertir a Chicago y Other

In [63]:
len(df)

215626

In [4]:
clean_col_names(df)

,inspection_id,dba_name,aka_name,license_#,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,results,violations,latitude,longitude,location
0,2473009,JENI'S SPLENDID ICE CREAMS,JENI'S SPLENDID ICE CREAMS,2767260.0,Restaurant,Risk 2 (Medium),2512 N MILKWAUKEE AVE,CHICAGO,IL,60647.0,01/22/2021,License,Pass,NaN,NaN,NaN,NaN
1,2457202,PI18EN,PI18EN,2626471.0,Restaurant,Risk 1 (High),1519 W 18TH ST,CHICAGO,IL,60608.0,11/30/2020,Non-Inspection,No Entry,NaN,41.857714,-87.664542,"(-87.66454238198166, 41.8577138739859)"
2,2453552,CLAMP DOWN BURGERS,NaN,2749943.0,NaN,Risk 2 (Medium),1742 W DIVISION ST,CHICAGO,IL,60622.0,10/20/2020,License,No Entry,NaN,41.903387,-87.671740,"(-87.67174026586648, 41.903386755553484)"
3,2401262,Fleming Branch,Fleming Branch,23462.0,School,Risk 1 (High),4918 W 64th ST,CHICAGO,IL,60638.0,09/16/2020,Canvass,Pass,NaN,41.776651,-87.745551,"(-87.74555107977419, 41.7766510612827)"
4,2387142,Pritzker,Pritzker,25871.0,School,Risk 1 (High),2009 W Schiller ST,CHICAGO,IL,60622.0,09/14/2020,Canvass,Pass,NaN,41.907025,-87.677796,"(-87.67779554588037, 41.907025004582245)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215621,215236,MANNY'S CAFE MIDWAY,MANNY'S CAFE MIDWAY,1197659.0,Restaurant,Risk 1 (High),5700 S CICERO AVE,CHICAGO,IL,60638.0,02/25/2010,Canvass,Pass,NaN,41.789329,-87.741646,"(-87.74164564419637, 41.789329323265385)"
215622,197282,WALGREENS #09001,WALGREENS,2003065.0,Grocery Store,Risk 3 (Low),191 N CLARK ST,CHICAGO,IL,60601.0,02/18/2010,License-Task Force,Pass,NaN,41.885491,-87.630822,"(-87.63082227177384, 41.88549078428762)"
215623,197256,CALIFORNIA PIZZA KITCHEN ASAP,CALIFORNIA PIZZA KITCHEN ASAP,1676087.0,Restaurant,Risk 1 (High),30 N LA SALLE ST,CHICAGO,IL,60602.0,02/08/2010,Out of Business,Fail,NaN,41.882695,-87.632531,"(-87.63253142666058, 41.88269466861121)"
215624,176413,JACKSON SQUARE NURSING & REHABILITATION CENTER,JACKSON SQUARE NURSING & REHABILITATION CENTER,47398.0,Long Term Care,Risk 1 (High),5130 W JACKSON BLVD,CHICAGO,IL,60644.0,02/26/2010,Canvass,Fail,18. NO EVIDENCE OF RODENT OR INSECT OUTER OPEN...,41.876771,-87.754771,"(-87.75477136468453, 41.87677135996502)"


## Categoricas para convertir a One Hot Encoding: 
 - risk
 - inspection_type
 - results
 - Facility_type (pasamos de 500 a 68 categorias antes de OHE)

## transformacion de fecha: 
 - inspection_date

In [5]:
# Results es nuestra label, no le queremos hacer nada? 
# O la queremos convertir en PASS vs. Fail y sacar todos los demás ? 


In [ ]:
## Crear una nueva variable a partir del barrio en que se encuentra? 
## O con el zip code está bien ? 

In [6]:
## Qué transformacion se le hace a la variable DATE ?

In [ ]:
## Violations, se le podría hacer un análisis de sentimiento? al tipo de comentario que se deja. .. y clasificarlo... 
##en una variable numérica


In [13]:
df.columns

Index(['inspection_id', 'dba_name', 'aka_name', 'license_#', 'facility_type',
       'risk', 'address', 'city', 'state', 'zip', 'inspection_date',
       'inspection_type', 'results', 'violations', 'latitude', 'longitude',
       'location'],
      dtype='object')

# Variables independientes muy correlacionadas: 

## dba y aka 

Estas dos variables son muy similares, con cuál nos quedamos ? 
(dos variables independientes muy correlacionadas entre sí)

In [180]:
df['aka_name'].isna().sum()

2488

In [181]:
df['dba_name'].isna().sum()


0

In [182]:
df['dba_name'].nunique()

28478

In [183]:
df['aka_name'].nunique()

27094

### Nos conviene quedarnos con dba_name por no tener nas y por tener más valores únicos. 

## Adress Longitude latitude location 

Estas tres variables están muy correlacionadas entre sí, dan la misma información habría que pensar con cuál quedarse.

## Inspection date 

## License_#

In [5]:
df['license_#'].count()

215609

In [6]:
df['license_#'].nunique()

39140

 - Hay más licencias que lugares, dba_name. 

In [ ]:
df.rename(columns={"A": "a", "B": "c"})

## Cambiar nombre de la variable: 

In [164]:
df.rename(columns= {'license_#':'license_number'}, inplace=True)

## debería ser una variable categórica: 

In [165]:
df['license_number'] = df.license_number.astype('category')



In [166]:
df.columns

Index(['inspection_id', 'dba_name', 'aka_name', 'license_number',
       'facility_type', 'risk', 'address', 'city', 'state', 'zip',
       'inspection_date', 'inspection_type', 'results', 'violations',
       'latitude', 'longitude', 'location'],
      dtype='object')

In [168]:
top100=df['license_number'].value_counts().sort_values(ascending=False)
top100

0.0          545
1354323.0    198
14616.0      176
1574001.0     83
1974745.0     59
            ... 
1948711.0      1
1948787.0      1
1948812.0      1
1948819.0      1
9999999.0      1
Name: license_number, Length: 39140, dtype: int64

In [169]:
type(top100)

pandas.core.series.Series

In [174]:
top100.to_frame()[:300]

,license_number
0.0,545
1354323.0,198
14616.0,176
1574001.0,83
1974745.0,59
...,...
1916161.0,24
1649172.0,24
1884173.0,24
2068942.0,24


### Facility type

### Son 500 categorias, pero veamos su proporcion: 

In [7]:
df['facility_type'].nunique()

500

In [8]:
## Elijo las top 67 categorias que son las que tienen al menos 20 valores:
top67=df['facility_type'].value_counts()[:67].sort_values(ascending=False)


In [9]:
top67

Restaurant                      143606
Grocery Store                    28146
School                           13220
Children's Services Facility      3733
Bakery                            3164
                                 ...  
GAS STATION/MINI MART               21
Live Poultry                        21
SUPPORTIVE LIVING                   21
Grocery(Sushi prep)                 20
ROOF TOP                            20
Name: facility_type, Length: 67, dtype: int64

In [125]:
## Pasamos de 500 categor'ias a 68 quedandonos con ls 67 más importantes y lo demás lo ponemos en other:
relevant_categories=top67.to_frame().index.values.tolist() 
df.loc[~df["facility_type"].isin(relevant_categories), "facility_type"] = "Other"


In [126]:
df['facility_type'].nunique()

68

## inspection_type

In [128]:
df['inspection_type'].nunique()

98

In [ ]:
df['inspection_type'] = df['inspection_type'].str.upper() 


In [ ]:
df['inspection_type'].nunique()

In [141]:
top25=df['inspection_type'].value_counts()[:25].sort_values(ascending=False)
top25

CANVASS                                   114518
LICENSE                                    28362
CANVASS RE-INSPECTION                      22875
COMPLAINT                                  19961
LICENSE RE-INSPECTION                       9650
COMPLAINT RE-INSPECTION                     8222
SHORT FORM COMPLAINT                        7274
SUSPECTED FOOD POISONING                     876
CONSULTATION                                 674
LICENSE-TASK FORCE                           604
TAG REMOVAL                                  603
RECENT INSPECTION                            367
OUT OF BUSINESS                              306
TASK FORCE LIQUOR 1475                       252
NON-INSPECTION                               249
SUSPECTED FOOD POISONING RE-INSPECTION       193
COMPLAINT-FIRE                               161
SHORT FORM FIRE-COMPLAINT                    113
NO ENTRY                                      72
SPECIAL EVENTS (FESTIVALS)                    63
PACKAGE LIQUOR 1474 

In [142]:
## Pasamos de 500 categor'ias a 68 quedandonos con ls 67 más importantes y lo demás lo ponemos en other:
relevant_categories_inspection=top25.to_frame().index.values.tolist() 
df.loc[~df["inspection_type"].isin(relevant_categories_inspection), "inspection_type"] = "Other"


In [143]:
df['inspection_type'].nunique()

26

# Convertir a mayusculas: 

### Se tienen 300 categorias menos en DBA pasando a mayuscula:

In [56]:
df['dba_name'].nunique()

28772

In [58]:
df['dba_name'].count()

215626

In [59]:
df['dba_name'] = df['dba_name'].str.upper() 


In [60]:
df['dba_name'].nunique()

28478

### Se obtienen casi 300 categorías menos llevandolo a mayuscula: 

In [57]:
df['aka_name'].nunique()

27380

In [61]:
df['aka_name'] = df['aka_name'].str.upper() 


In [62]:
df['aka_name'].nunique()

27094

In [ ]:
df['inspection_type'] = df['inspection_type'].str.upper() 


In [26]:
df['inspection_type'].unique()

## Convertir todo a mayuscula. 

array(['License', 'Non-Inspection', 'Canvass', 'Canvass Re-Inspection',
       'Complaint', 'Short Form Complaint', 'Complaint Re-Inspection',
       'License Re-Inspection', 'Suspected Food Poisoning',
       'Recent Inspection', nan, 'Consultation', 'Complaint-Fire',
       'Short Form Fire-Complaint', 'License-Task Force',
       'Complaint-Fire Re-inspection',
       'Suspected Food Poisoning Re-inspection', 'Package Liquor 1474',
       'Out of Business', 'Tag Removal', 'Task Force Liquor 1475',
       'No Entry', 'LICENSE REQUEST', 'out ofbusiness',
       'TASK FORCE LIQUOR (1481)', 'TASK FORCE LIQUOR 1470',
       'OFFICE ASSIGNMENT', 'COVID COMPLAINT', 'Not Ready',
       'Recent inspection', 'Special Events (Festivals)',
       'fire complaint', 'FIRE', 'KITCHEN CLOSED FOR RENOVATION', 'O.B.',
       'License consultation', 'CORRECTIVE ACTION',
       'LICENSE CONSULTATION', 'LICENSE CANCELED BY OWNER',
       'OWNER SUSPENDED OPERATION/LICENSE', 'Illegal Operation',
       '

In [22]:
df['state'].unique()

array(['IL', nan, 'NY', 'WI', 'IN'], dtype=object)

In [23]:
df['city'].unique()

array(['CHICAGO', 'Chicago', 'SCHAUMBURG', 'ALGONQUIN', 'ELMHURST',
       'CHICAGO.', 'WILMETTE', 'chicago', nan, 'ELK GROVE VILLAGE',
       'BLOOMINGDALE', 'CICERO', 'SKOKIE', 'JUSTICE', 'TINLEY PARK',
       'INACTIVE', 'DES PLAINES', 'LAKE ZURICH', 'WORTH', 'ALSIP',
       'CCHICAGO', 'OAK PARK', 'CHICAGOCHICAGO', 'EVANSTON',
       'CHICAGOHICAGO', '312CHICAGO', 'CHicago', 'BERWYN',
       'chicagoBEDFORD PARK', 'BRIDGEVIEW', 'NEW YORK', 'NEW HOLSTEIN',
       'NILES NILES', 'GRIFFITH', 'WADSWORTH', 'LANSING', 'WHEATON',
       'ROSEMONT', 'CALUMET CITY', 'HIGHLAND PARK', 'PLAINFIELD',
       'PALOS PARK', 'MAYWOOD', 'LAKE BLUFF', 'SCHILLER PARK',
       'BANNOCKBURNDEERFIELD', 'CHCICAGO', 'Norridge', 'CHARLES A HAYES',
       'CHICAGOI', 'CHCHICAGO', 'COUNTRY CLUB HILLS', 'CHICAGO HEIGHTS',
       'SUMMIT', 'LOMBARD', 'WESTMONT', 'OOLYMPIA FIELDS',
       'EAST HAZEL CREST', 'BOLINGBROOK', 'STREAMWOOD', 'BURNHAM',
       'OAK LAWN', 'OLYMPIA FIELDS', 'alsip', 'GLENCOE', 'NAPERVI

## Transforma City con regex

In [34]:
# esta transformacion elimina 12 categorías de CITY chicago, que estaban diferentemente escritas 
# Pasan de 70 a 58 en nuestro data set: 

df.loc[df['city'].str.contains('chic', case=False, na=False), 'city'] = 'CHICAGO'


In [35]:
df['city'].nunique()

58

In [36]:
df['city'].unique()

array(['CHICAGO', 'SCHAUMBURG', 'ALGONQUIN', 'ELMHURST', 'WILMETTE', nan,
       'ELK GROVE VILLAGE', 'BLOOMINGDALE', 'CICERO', 'SKOKIE', 'JUSTICE',
       'TINLEY PARK', 'INACTIVE', 'DES PLAINES', 'LAKE ZURICH', 'WORTH',
       'ALSIP', 'OAK PARK', 'EVANSTON', 'BERWYN', 'BRIDGEVIEW',
       'NEW YORK', 'NEW HOLSTEIN', 'NILES NILES', 'GRIFFITH', 'WADSWORTH',
       'LANSING', 'WHEATON', 'ROSEMONT', 'CALUMET CITY', 'HIGHLAND PARK',
       'PLAINFIELD', 'PALOS PARK', 'MAYWOOD', 'LAKE BLUFF',
       'SCHILLER PARK', 'BANNOCKBURNDEERFIELD', 'CHCICAGO', 'Norridge',
       'CHARLES A HAYES', 'COUNTRY CLUB HILLS', 'SUMMIT', 'LOMBARD',
       'WESTMONT', 'OOLYMPIA FIELDS', 'EAST HAZEL CREST', 'BOLINGBROOK',
       'STREAMWOOD', 'BURNHAM', 'OAK LAWN', 'OLYMPIA FIELDS', 'alsip',
       'GLENCOE', 'NAPERVILLE', 'BROADVIEW', 'BLUE ISLAND', 'FRANKFORT',
       'Maywood', 'EVERGREEN PARK'], dtype=object)

### Reduce 12 categorías que eran lo mismo pero estaban en mayuscula o minuscula: 

# Pipeline para seleccion de variables

In [187]:
df.head(5)

,inspection_id,dba_name,aka_name,license_number,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,results,violations,latitude,longitude,location
0,2473009,JENI'S SPLENDID ICE CREAMS,JENI'S SPLENDID ICE CREAMS,2767260.0,Restaurant,Risk 2 (Medium),2512 N MILKWAUKEE AVE,CHICAGO,IL,60647.0,01/22/2021,LICENSE,Pass,NaN,NaN,NaN,NaN
1,2457202,PI18EN,PI18EN,2626471.0,Restaurant,Risk 1 (High),1519 W 18TH ST,CHICAGO,IL,60608.0,11/30/2020,NON-INSPECTION,No Entry,NaN,41.857714,-87.664542,"(-87.66454238198166, 41.8577138739859)"
2,2453552,CLAMP DOWN BURGERS,NaN,2749943.0,Other,Risk 2 (Medium),1742 W DIVISION ST,CHICAGO,IL,60622.0,10/20/2020,LICENSE,No Entry,NaN,41.903387,-87.671740,"(-87.67174026586648, 41.903386755553484)"
3,2401262,FLEMING BRANCH,FLEMING BRANCH,23462.0,School,Risk 1 (High),4918 W 64th ST,CHICAGO,IL,60638.0,09/16/2020,CANVASS,Pass,NaN,41.776651,-87.745551,"(-87.74555107977419, 41.7766510612827)"
4,2387142,PRITZKER,PRITZKER,25871.0,School,Risk 1 (High),2009 W Schiller ST,CHICAGO,IL,60622.0,09/14/2020,CANVASS,Pass,NaN,41.907025,-87.677796,"(-87.67779554588037, 41.907025004582245)"


## sklearn imports

In [237]:
from sklearn.preprocessing import (
    OneHotEncoder,
    StandardScaler
)
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import (
    GridSearchCV,
    TimeSeriesSplit
)
from sklearn.model_selection import (
    train_test_split,
    cross_val_score
)
from sklearn.ensemble import RandomForestClassifier


In [245]:
sys.path.append("/../")

In [246]:
pwd

'/Users/sansansansan/repos_itam/dpa_2021/notebooks'

## por que le tengo que poner el _ml para que funcione el IMPORT __?? 

In [252]:
from src.utils.params_ml import time_series_splits, evaluation_metric

## One Hot Encoding 

In [205]:
df_features= df.loc[:, df.columns != 'results']

In [228]:
df_labels= df.loc[:, df.columns == 'results']

In [200]:
categoric_pipeline = Pipeline([
    ('hotencode',OneHotEncoder())
])

In [202]:
## elijo los features 
cat_features = ['risk','inspection_type','facility_type']

In [207]:
pipeline = ColumnTransformer(transformers=[('categoric', categoric_pipeline, cat_features)],
        remainder='passthrough', sparse_threshold=0)


In [208]:
df_features_prc = pipeline.fit_transform(df_features)

In [209]:
type(df_features_prc)

numpy.ndarray

In [211]:
len(df_features_prc)

215626

In [ ]:
np.shape(df_features_prc)

In [213]:
import numpy as np

(215626, 112)

In [215]:
df_features_prc

array([[0.0, 0.0, 1.0, ..., nan, nan, nan],
       [0.0, 1.0, 0.0, ..., 41.8577138739859, -87.66454238198166,
        '(-87.66454238198166, 41.8577138739859)'],
       [0.0, 0.0, 1.0, ..., 41.90338675555349, -87.67174026586648,
        '(-87.67174026586648, 41.903386755553484)'],
       ...,
       [0.0, 1.0, 0.0, ..., 41.88269466861121, -87.63253142666058,
        '(-87.63253142666058, 41.88269466861121)'],
       [0.0, 1.0, 0.0, ..., 41.87677135996502, -87.75477136468453,
        '(-87.75477136468453, 41.87677135996502)'],
       [0.0, 1.0, 0.0, ..., 41.90011845725853, -87.62398051946951,
        '(-87.62398051946951, 41.90011845725853)']], dtype=object)

In [ ]:
## Selecting and training model - Random Forrest
#model = models_dict["random_forest"]["model"]

In [229]:
X_train, X_test, y_train, y_test = train_test_split(df_features_prc, df_labels, test_size=0.3)

In [233]:
models_dict = {
    "random_forest": {
        "model": RandomForestClassifier(
            max_features=6,
            n_estimators=10,
            max_leaf_nodes=10,
            oob_score=True,
            n_jobs=-1,
            random_state=1111),
        "param_grid": {
            "n_estimators": [100, 300, 500, 800],
            "min_samples_leaf": [3, 5, 7],
            "criterion": ['gini']
        }
    }
}


In [235]:
model = models_dict["random_forest"]["model"]
print("\n++ The model that will be used is: {}\n".format(model))


++ The model that will be used is: RandomForestClassifier(max_features=6, max_leaf_nodes=10, n_estimators=10,
                       n_jobs=-1, oob_score=True, random_state=1111)



In [253]:
grid_search = GridSearchCV(model,
                           models_dict["random_forest"]["param_grid"],
                           cv=TimeSeriesSplit(n_splits=time_series_splits),
                           scoring=evaluation_metric,
                           return_train_score=True,
                           n_jobs=-1
                           )

In [254]:
grid_search.fit(X_train, y_train)

/Users/sansansansan/.pyenv/versions/3.7.4/envs/dpa/lib/python3.7/site-packages/sklearn/model_selection/_search.py:921: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan]
  category=UserWarning
/Users/sansansansan/.pyenv/versions/3.7.4/envs/dpa/lib/python3.7/site-packages/sklearn/model_selection/_search.py:921: UserWarning: One or more of the train scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan]
  category=UserWarning


ValueError: could not convert string to float: 'PINKBERRY - STATE STREET SERIES'

In [ ]:
print("\n++ Grid search results:\n")
print("    ++++ Best parameters: {}".format(grid_search.best_params_))
print("    ++++ Best estimator: {}".format(grid_search.best_estimator_))
print("    ++++ Number of features in best estimator: {} \n".format(grid_search.best_estimator_.n_features_))
print("    ++++ Best estimator score: {}\n".format(grid_search.best_score_))
print("    ++++ Best estimator oob score: {}\n".format(grid_search.best_estimator_.oob_score_))